In [2]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [3]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Fil/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Fil/metadata_105.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,file2_2.wav,1,105,Al-Fil
1,file2_3.wav,1,105,Al-Fil
2,file2_4.wav,1,105,Al-Fil
3,file2_5.wav,1,105,Al-Fil
4,file2_6.wav,1,105,Al-Fil
5,file3_2.wav,2,105,Al-Fil
6,file3_3.wav,2,105,Al-Fil
7,file3_4.wav,2,105,Al-Fil
8,file3_5.wav,2,105,Al-Fil
9,file3_6.wav,2,105,Al-Fil


Feature Extraction

In [4]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [5]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [6]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00217523  0.00521813  0.00339093 ... -0.29806244 -0.43185666
 -0.24938205] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.14031436 -0.14093682 -0.09778552 ...  0.47530606  0.40005422
  0.55055577] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.19895402 -0.2985239  -0.24242662 ... -0.16960028 -0.31309488
 -0.39246798] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.539929   0.4481

In [7]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (5, 704, 40)
Shape of labels for fold 1: (5,)


In [8]:
mfcc_data_by_fold[3]["data"].shape

(5, 704, 40)

In [9]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [10]:
mfcc_data_by_fold[1]["data"].shape

(5, 704, 40, 1)

After Training Work, Deep Features 

In [11]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 63ms/step


In [12]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [13]:
intermediate_layer_outputs.shape

(3, 5, 1024)

In [14]:
intermediate_layer_outputs

array([[[-0.9966796 ,  0.9913081 ,  0.9997488 , ...,  0.9984734 ,
         -0.962979  , -0.99998736],
        [-0.93324685,  0.76708627,  0.99977   , ...,  0.99805427,
         -0.5641571 , -0.9984994 ],
        [-0.97005504,  0.99884194,  0.99764955, ...,  0.9967407 ,
         -0.7019466 , -0.9999055 ],
        [-0.9935172 ,  1.        ,  0.99997896, ...,  0.99583876,
         -1.        , -0.9577738 ],
        [-0.89148533,  0.58337855,  0.9971279 , ...,  0.99977314,
         -0.76066726, -0.9997212 ]],

       [[-0.9945564 , -1.        ,  1.        , ...,  0.95468074,
         -0.9799491 , -0.9999792 ],
        [-0.99746776, -1.        ,  0.99999076, ...,  0.99520576,
          0.17624061, -0.9999809 ],
        [-0.9999667 , -1.        ,  1.        , ...,  0.9760541 ,
         -0.99677   , -0.9999979 ],
        [-0.9983352 ,  1.        ,  1.        , ...,  0.87811196,
         -1.        , -0.9999962 ],
        [-0.99999803, -1.        ,  1.        , ...,  0.99998754,
         -0.85

In [15]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [16]:
al_fil_mean = mean_across_folds

In [17]:
al_fil_mean.shape

(5, 1024)

In [27]:
al_fil_mean

array([[-0.51423717,  0.33039936,  0.9994244 , ...,  0.98360246,
        -0.9809732 , -0.99970514],
       [-0.9760046 ,  0.03995533,  0.9990788 , ...,  0.9964437 ,
        -0.46263465, -0.9994085 ],
       [-0.923538  ,  0.16588955,  0.9973097 , ...,  0.9895962 ,
        -0.8975327 , -0.99960446],
       [-0.9972306 ,  0.99766463,  0.9999883 , ...,  0.95794374,
        -0.99999887, -0.98584944],
       [-0.9543843 , -0.4626484 ,  0.99617213, ...,  0.99746853,
        -0.86506677, -0.9872804 ]], dtype=float32)

In [28]:
# Specify the file path where you want to save the .npy file
file_path = 'al_fil_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_fil_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/testing4.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.09312387 -0.16650012 -0.1283591  ...  0.13541992  0.08281836
 -0.15978362] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.47740942  0.5156708   0.51956135 ... -0.886065   -1.0185649
 -0.99103105] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.2417877   0.06852865  0.35304728 ...  0.40275878  0.32651764
  0.29308468] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.5820711   0.08506176 -0.24607421 

In [20]:
mfcc_extracted_features_single

array([[[[-7.6138634e+01],
         [ 1.1789563e+02],
         [-2.4836512e+01],
         ...,
         [-1.1109576e+00],
         [-3.8540106e+00],
         [ 1.7562889e+00]],

        [[-1.8079348e+01],
         [ 8.9505051e+01],
         [-2.4861050e+01],
         ...,
         [-7.3070498e+00],
         [-6.7992964e+00],
         [ 6.4401622e+00]],

        [[ 1.2121010e+01],
         [ 6.0270958e+01],
         [-3.3406509e+01],
         ...,
         [-1.2088049e+01],
         [-1.2273276e+01],
         [ 9.1229563e+00]],

        ...,

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
 

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 108ms/step


In [22]:
intermediate_output

array([[-0.3269354 ,  0.20568976,  0.9460579 , ...,  0.95006675,
        -0.97158957, -0.99939054]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.9966796 ,  0.9913081 ,  0.9997488 , ...,  0.9984734 ,
         -0.962979  , -0.99998736],
        [-0.93324685,  0.76708627,  0.99977   , ...,  0.99805427,
         -0.5641571 , -0.9984994 ],
        [-0.97005504,  0.99884194,  0.99764955, ...,  0.9967407 ,
         -0.7019466 , -0.9999055 ],
        [-0.9935172 ,  1.        ,  0.99997896, ...,  0.99583876,
         -1.        , -0.9577738 ],
        [-0.89148533,  0.58337855,  0.9971279 , ...,  0.99977314,
         -0.76066726, -0.9997212 ]],

       [[-0.9945564 , -1.        ,  1.        , ...,  0.95468074,
         -0.9799491 , -0.9999792 ],
        [-0.99746776, -1.        ,  0.99999076, ...,  0.99520576,
          0.17624061, -0.9999809 ],
        [-0.9999667 , -1.        ,  1.        , ...,  0.9760541 ,
         -0.99677   , -0.9999979 ],
        [-0.9983352 ,  1.        ,  1.        , ...,  0.87811196,
         -1.        , -0.9999962 ],
        [-0.99999803, -1.        ,  1.        , ...,  0.99998754,
         -0.85

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_fil_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_fil_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_fil_mean: {closest_index+1}")
print(f"Closest value in al_fil_mean: {closest_value}")

Index of closest value in al_fil_mean: 3
Closest value in al_fil_mean: [-0.923538    0.16588955  0.9973097  ...  0.9895962  -0.8975327
 -0.99960446]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")